In [1]:
from diffusers import UNet2DModel, DDIMScheduler, VQModel
import torch
import PIL.Image
import numpy as np
import tqdm

torch.__version__

'1.13.1+cu117'

In [2]:
# load all models
unet = UNet2DModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="unet")
vqvae = VQModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")

# set to cuda
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

unet.to(torch_device)
vqvae.to(torch_device);

/home/lcur1728/.local/lib/python3.8/site-packages/diffusers/configuration_utils.py:215: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [3]:
seed = 3

# generate gaussian noise to be decoded
generator = torch.manual_seed(seed)
noise = torch.randn(
    (1, unet.in_channels, unet.sample_size, unet.sample_size),
    generator=generator,
).to(torch_device)

# set inference steps for DDIM
scheduler.set_timesteps(num_inference_steps=200)

<ipython-input-3-721d8f03a340>:6: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DModel's config object instead, e.g. 'unet.config.in_channels'.
  (1, unet.in_channels, unet.sample_size, unet.sample_size),


In [4]:
image = noise
frames_unsampled = []
for t in tqdm.tqdm(scheduler.timesteps):
    # predict noise residual of previous image
    with torch.no_grad():
        residual = unet(image, t)["sample"]

    # compute previous image x_t according to DDIM formula
    prev_image = scheduler.step(residual, t, image, eta=0.0)["prev_sample"]

    # x_t-1 -> x_t
    image = prev_image

    # decode image with vae
    with torch.no_grad():
        image_frame = vqvae.decode(image)
        frames_unsampled.append(image_frame)

100%|██████████| 200/200 [00:15<00:00, 12.63it/s]


In [7]:
from PIL import Image, ImageSequence, ImageDraw, ImageFont

frames = []

step = 0
for frame in frames_unsampled:
    # process image
    image_processed = frame.sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.clamp(0, 255).numpy().astype(np.uint8)
    image_pil = PIL.Image.fromarray(image_processed[0])
    text = f"Step = {step}"
    ImageDraw.Draw(image_pil).text((10, 10), text, (0, 0, 0))
    # draw = ImageDraw.Draw(image_pil)
    # text = f"Step = {step}"
    # font = ImageFont.truetype("sans-serif.ttf", 16)
    # draw.text((10, 10), text, (255,255,255), font=font)
    frames.append(image_pil)
    step += 1

frames = frames + [frames[-1]] * 1000
frames[0].save(f"../../figures/ablation/ldm_decoded_latent_{seed}.gif", format="GIF", append_images=frames[1:], save_all=True, duration=0.5, loop=0)

OSError: cannot open resource

: 